In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

<p> First, we load the dataset and apply some pre-processing techniques shown in previous sections </p>

In [10]:
df = pd.read_csv("Dataset - Group 50 - Data - New.csv")
df = df.iloc[:158]
df.drop(['Add columns here', 'Add columns here.1', 'Add columns here.2', 'Reviewer', 'Review'], axis = 1, inplace = True)
df.drop(['Account bio', 'Tweet Translated', 'Views', 'Remarks', 'Location', 'Reasoning', 'Screenshot'], axis = 1, inplace = True)
df['Quote Tweets'].fillna(0, inplace = True)
df['Replies'].fillna(0, inplace = True)
df['Rating'].fillna('FALSE', inplace = True)
df['Date posted'] = pd.to_datetime(df['Date posted'])
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df['Rating'] = df['Rating'].replace('FALSE. MISLEADING', 'FALSE, MISLEADING')
df['Followers'] = df['Followers'].replace('28.3K', '28,300')
df['Followers'] = df['Followers'].replace('20.4K', '20,300')

C:\Users\Fort Bueno\AppData\Local\Temp\ipykernel_14036\2151456633.py:8: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

C:\Users\Fort Bueno\AppData\Local\Temp\ipykernel_14036\2151456633.py:9: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



In [11]:
from calendar import month_name as mn
import matplotlib.dates as mdates
import numpy as np

grouped_df = df.groupby('Date posted').size().reset_index(name='count')
grouped_df.set_index(['Date posted'], inplace = True)
count_monthly = grouped_df[['count']].resample('M').sum()  #resample by months
count_monthly.reset_index(inplace=True)

count_monthly['count'] = count_monthly['count'].replace(0, np.NaN)
count_monthly['count_linear'] = count_monthly['count'].interpolate(method='linear')
count_monthly['count'] = count_monthly['count'].replace(np.NaN, 0)

count_monthly.head()

,Date posted,count,count_linear
0,2016-03-31,1.0,1.0
1,2016-04-30,1.0,1.0
2,2016-05-31,0.0,1.0
3,2016-06-30,0.0,1.0
4,2016-07-31,0.0,1.0


In [12]:
## Perform peak detection
from scipy.signal import find_peaks
import ruptures as rpt
import plotly.graph_objects as go

x = count_monthly['Date posted'].astype(np.int64) / 10**9  # Convert to seconds (UNIX epoch start)
x = x.values.reshape(-1, 1) 
y = count_monthly['count_linear']

peaks, _ = find_peaks(count_monthly['count_linear'],
                      height=5,  # height of peaks
                      width=0.5,       # width of peaks
                      threshold=1, # vertical distance to its neighboring samples
                      distance=7,   # minimal horizontal distance (>= 1) in samples between neighbouring peaks
                      prominence=1) # vertical distance between the peak and its lowest contour line

data = count_monthly['count_linear'].values

# Extract event timestamps
events = count_monthly.iloc[peaks]

# Change-point detection via Pelt algorithm
model = "rbf"
algo = rpt.Pelt(model=model).fit(data)
result = algo.predict(pen=.8)  # Adjust the penalty value based on your data

# Plot peaks
fig = go.Figure()

# Original data
fig.add_trace(go.Scatter(
  x=count_monthly.index,
  y=count_monthly['count_linear'],
  hovertext=count_monthly['Date posted'].dt.strftime('%Y-%m-%d'),
  mode='lines',
  name='Original Data',
  line=dict(
    color='#f3f3f3',
    width = 4
  )))

for cp_index in result[:-1]:
  x_1 = count_monthly.index[cp_index]
  y_1, y_2, y_3, y_4, y_5 = -2, 0, 2, 4, 5
  y_6, y_7, y_8, y_9, y_10 = 7, 10, 14, 18, 22

  fig.add_trace(go.Scatter(
    x=[x_1, x_1, x_1, x_1, x_1, x_1, x_1, x_1, x_1, x_1],
    y=[y_1, y_2, y_3, y_4, y_5, y_6, y_7, y_8, y_9, y_10],
    mode='lines',
    name=count_monthly['Date posted'].dt.strftime('%Y-%m-%d')[cp_index],
    hoverinfo='text',
    hovertext='Change at<br>%s'%count_monthly['Date posted'].dt.strftime('%Y-%m-%d')[cp_index],
    line=dict(color='#a5ffd9', width=4, dash='dash')))


# Peaks
fig.add_trace(go.Scatter(
  x=events.index,
  y=events['count_linear'],
  hovertext=events['Date posted'].dt.strftime('%Y-%m-%d'),
  mode='markers',
  name='Peaks',
  marker=dict(
    color='#ff5e84',
    size=10,
    symbol='x')))

fig.update_layout(height=600,
                  title='<span style = "color:#f3f3f3"><b>Twitter Risa Tweet Mis/Disinformation Philhealth Allegations Peaks & Change Points</b></span>',
                  xaxis=dict(title='<b>Date</b>', color = "#f3f3f3"),
                  yaxis=dict(title='<b>Tweet Count</b>', color = "#f3f3f3"),
                  yaxis_range=[-2, 22],
                  plot_bgcolor="#2A3035",
                  paper_bgcolor="#31373E",
                  font_color = "#f3f3f3")

xtv = np.arange(0,82,16)
xtt = count_monthly['Date posted'].dt.strftime('%Y-%m-%d')[xtv]
fig.update_xaxes(tickmode='array',
                 tickvals=xtv,
                 ticktext=xtt)

fig.show()